In [67]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import re
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np


In [110]:
# Leer el archivo CSV
df = pd.read_csv('C:/Users/Hernán Ifrán/Downloads/resultados.csv', encoding='utf-8')  
# Obtener las columnas de diagnósticos y códigos AIS
descripciones = df['Description'].tolist()
codigos_AIS = df['Codigo_AIS'].tolist()

descripciones = [str(desc) for desc in descripciones]

# Codificar los códigos AIS como etiquetas numéricas
label_encoder = LabelEncoder()
codigos_AIS_numericos = label_encoder.fit_transform(codigos_AIS)

In [111]:
# Función para separar diagnósticos en una descripción
def separar_diagnosticos(descripcion):
    pattern = r'\s*\+\s*|\s+(?<!\w)\.\s+|\n'
   # pattern = r'\s*\+\s*|\s+(?<!\w)\.\s+|\s*,\s*|\n'
    return re.split(pattern, descripcion.strip())

# Función para realizar stemming en una descripción
stemmer = SnowballStemmer('spanish')
def stem_descripcion(descripcion):
    words = nltk.word_tokenize(descripcion)
    words = [stemmer.stem(word) for word in words if word not in stopwords.words('spanish')]
    return ' '.join(words)

# Aplicar stemming a las descripciones
descripciones_stemmed = [stem_descripcion(desc) for desc in descripciones]

In [112]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(descripciones_stemmed, codigos_AIS_numericos, test_size=0.15, random_state=42)

# Convertir las descripciones de texto a representaciones numéricas
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Calcular el número de muestras en cada clase
class_counts = dict(zip(*np.unique(y_train, return_counts=True)))

# Definir la proporción de sobremuestreo para cada clase
smote = SMOTE(sampling_strategy={0: 150, 1: 200, 2: 170})  # Ajusta según tus necesidades

# Aplicar SMOTE a las características numéricas
X_resampled, y_resampled = smote.fit_resample(X_train_tfidf, y_train)


In [113]:
# Creación del pipeline para el clasificador basado en texto
pipeline = make_pipeline(
  
    RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2)  
)

# Entrenamiento del modelo
pipeline.fit(X_resampled, y_resampled)

Pipeline(steps=[('randomforestclassifier', RandomForestClassifier())])

In [ ]:
# Guardar el modelo
modelo_path = 'C:/Users/Hernán Ifrán/Downloads/modelo_entrenado.joblib'
joblib.dump(pipeline, modelo_path)
print(f"El modelo se ha guardado correctamente en: {modelo_path}")

In [119]:
# Evaluación del modelo
X_test_tfidf = tfidf_vectorizer.transform(X_test)  # Transformar los datos de prueba
y_pred = pipeline.predict(X_test_tfidf)
report = classification_report(y_test, y_pred, zero_division=1)
print(report)


              precision    recall  f1-score   support

           0       0.47      0.60      0.53        25
           1       0.33      0.80      0.47         5
           2       0.20      0.20      0.20         5
           3       0.50      0.40      0.44         5
           4       1.00      0.50      0.67         2
           6       0.67      0.50      0.57         4
           7       1.00      0.00      0.00         2
           8       0.00      1.00      0.00         0
           9       0.92      0.92      0.92        12
          10       1.00      0.33      0.50         3
          12       1.00      0.40      0.57        10
          13       1.00      0.00      0.00         1
          16       0.88      0.94      0.91        48
          17       0.92      0.99      0.96       229
          18       1.00      0.79      0.88        14
          19       0.99      1.00      0.99        89
          20       0.92      0.96      0.94       135
          21       1.00    

In [ ]:
# Evaluación del modelo con validación cruzada
cv_scores = cross_val_score(pipeline, descripciones_stemmed, codigos_AIS_numericos, cv=5, scoring='accuracy')
print(f'Accuracy promedio en validación cruzada: {cv_scores.mean():.2f}')

In [116]:
archivo_diagnosticos = 'C:/Users/Hernán Ifrán/Downloads/pruebadiagnosticos.txt'
with open(archivo_diagnosticos, 'r',encoding='latin-1') as file:
    texto_diagnosticos = file.read()

In [117]:

#archivo_diagnosticos = 'C:/Users/Hernán Ifrán/Downloads/pruebadiagnosticos.txt'
#with open(archivo_diagnosticos, 'r',encoding='utf-8') as file:
#    texto_diagnosticos = file.read()


diagnosticos_transformados = tfidf_vectorizer.transform(diagnosticos)
predicciones = pipeline.predict(diagnosticos_transformados)    
clases_predichas = label_encoder.inverse_transform(predicciones)
#diagnosticos = separar_diagnosticos(texto_diagnosticos)
#for diagnostico in diagnosticos:
for diagnostico, clase_predicha in zip(diagnosticos, clases_predichas):
    #diagnostico_stemmed = stem_descripcion(diagnostico.strip())
    #predicciones = pipeline.predict([diagnostico_stemmed])
    #predicciones = pipeline.predict(diagnosticos_transformados)
    #clases_predichas = label_encoder.inverse_transform(predicciones)
    print('Descripción:', diagnostico.strip())
    print('Códigos AIS predichos:', clases_predichas)
    print('---')



Descripción: EscoriaciÃ³n regiÃ³n sacra
Códigos AIS predichos: [410402.1 410402.1 751111.2]
---
Descripción: Fractura de C5 con desplazamiento
Códigos AIS predichos: [410402.1 410402.1 751111.2]
---
Descripción: FX de calcaneo
Códigos AIS predichos: [410402.1 410402.1 751111.2]
---


In [ ]:
# Guardar el LabelEncoder en la carpeta Downloads
label_encoder_path = 'C:/Users/Hernán Ifrán/Downloads/label_encoder.joblib'
joblib.dump(label_encoder, label_encoder_path)
print(f"El LabelEncoder se ha guardado correctamente en: {label_encoder_path}")